### 練習問題2.19
1.2.2節の両替パターン数計算プログラムについ て考える。  
プログラムで使う通貨を変えられるようにして、  
例えばイギリスポンドの両替パターン数を計算できるようになると便利だろう。  
現状のプログラムでは、通貨についての知識は⼀部は first-denominationに、  
⼀部はcount-change (アメリカには5種 類のコインがあるということを知っている)に分散されている。  
両替に使うコインのリストを与えられるようになればもっとよくなるだろう。  
⼿続きccを書き直して、⼆つ⽬の引数として取るものを、どのコインを使うかを指定する整数ではなく、  
使うコインの値のリスト となるようにしたい。  
それから、それぞれの通貨の種類を定義するリストを次のように持っておく。  

    (define us-coins (list 50 25 10 5 1))
    (define uk-coins (list 100 50 20 10 5 2 1 0.5))
    
そして、ccは次のように呼び出すことにしよう。  

    (cc 100 us-coins)
    292
    
このためには、プログラムccに対して何らかの変更を加える必要がある。  
プログラムの形は同じだが、⼆つ⽬の引数に対するアクセスの仕⽅が次のように変わる。

    (define (cc amount coin-values)
        (cond ((= amount 0) 1)
              ((or (< amount 0) (no-more? coin-values)) 0)
              (else
                  (+ (cc amount
                      (except-first-denomination coin-values))
                      (cc (- amount (first-denomination
                                      coin-values))
                          coin-values)))))
                      
リスト構造に対する基本演算によって、  
⼿続きfirst-denomination, except-first-denomination, no-more?を定義せよ。  
リストcoinvaluesの順序は、ccによって返される解答に影響を与えるだろう か、  
それとも与えないだろうか。それはどうしてだろうか。

In [1]:
(define (cc amount coin-values)
    (cond ((= amount 0) 1)
          ((or (< amount 0) (no-more? coin-values)) 0)
          (else
              (+ (cc amount (except-first-denomination coin-values))
                  (cc (- amount (first-denomination coin-values)) coin-values))
           )
    )
  )

; coin-values(リスト)が空どうかを判定する。
(define (no-more? coin-values)(null? coin-values))
; 最初のコイン種別を除いたコイン種別リストを返す。
(define (except-first-denomination coin-values)(cdr coin-values))
; 最初のコイン種別を返す。
(define (first-denomination coin-values)(car coin-values))

In [2]:
; 動作確認
(define us-coins (list 50 25 10 5 1))
(display us-coins)
(newline)
;(display us-coins)
(display (cc 11 us-coins))
(newline)
(display (cc 50 us-coins))
(newline)
;(display (cc 100 us-coins))
;(newline)

; 順番を変えた場合の動作確認
; 合計金額が両替金額になった場合、組み合わせの数を+1するので、
; コインリストの順番を変えても出力結果は変わらない。
(define us-coins-r (reverse us-coins))
(display us-coins-r)
(newline)
(display (cc 11 us-coins))
(newline)
(display (cc 50 us-coins))
(newline)

(50 25 10 5 1)
4
50
(1 5 10 25 50)
4
50


練習問題とは別に、  
コイン両替の組み合わせの動作確認をするために、  
以下の出力イメージで表示させたい。

    1:((50 . 0)(25 . 0)(10 . 1)(5 . 0)(1 . 1))
    2:((50 . 0)(25 . 0)(10 . 0)(5 . 2)(1 . 1))
    3:((50 . 0)(25 . 0)(10 . 0)(5 . 1)(1 . 6))
    4:((50 . 0)(25 . 0)(10 . 0)(5 . 0)(1 . 11))

In [3]:
; 以下、補助手続き
(define (make-pair type val)(cons type val))
(define (coin-type p)(car p))
(define (coin-val p)(cdr p))

; 内容を更新した両替リストを新たに生成して返す。
(define (update-list l p)
  (if (null? l) '()
      (let ((t (car l))
            (remain (cdr l))
            (new-type (coin-type p))
            (new-val (coin-val p))
            )
          (let ((type (coin-type t))
                (val (coin-val t))
                )
            (if (= new-type type)
                    (let ((new-p (make-pair new-type (+ new-val val))))
                      (cons new-p (update-list remain p))
                      )
                (cons t (update-list remain p))
                )
            )
        )
      )
  )
; リストを更新する場合は、以下のアルゴリズムは正常に動作しない。
;(define (update-list l p)
;  (let 
;    ((type (coin-type p))
;     (val (coin-val p)))
;    (let ((result (assv type l)))
;      (if (eq? result #f) (append l (list p))
;          (let ((new-val (+ (coin-val result) val)))
;            (begin
;             (set-cdr! result new-val)
;             l
;             )
;            )
;          )
;      )
;    )
;  )

(display (update-list (list (make-pair 10 1) (make-pair 3 2) (make-pair 1 3)) (make-pair 5 7)))
(newline)
(display (update-list (list (make-pair 10 1) (make-pair 5 2) (make-pair 1 3)) (make-pair 5 7)))
(newline)

(define (make-list coin-values)
  (if (null? coin-values) '()
      (let ((p (make-pair (car coin-values) 0)))
          (cons p (make-list (cdr coin-values)))
        )
    )
  )
(define null-list (list))
(display null-list)
(newline)
(define us-coins (list 50 25 10 5 1))
(display us-coins)
(newline)
(display (make-list us-coins))
(newline)
;(display (length (make-list us-coins)))
;(newline)
(define (print-list l)
  (define (iter l-tmp cnt)
    (if (null? l-tmp) '()
        (let ((item (car l-tmp)))
          (begin
           (display cnt)
           (display ":")
           (display item)
           (newline)
           (iter (cdr l-tmp) (+ cnt 1))
           )
        )
      )
    )
    (iter l 1)
  )
;(print-list (make-list us-coins))
;(newline)
(print-list (update-list (list (make-pair 10 1) (make-pair 5 2) (make-pair 1 3)) (make-pair 5 7)))

((10 . 1) (3 . 2) (1 . 3))
((10 . 1) (5 . 9) (1 . 3))
()
(50 25 10 5 1)
((50 . 0) (25 . 0) (10 . 0) (5 . 0) (1 . 0))
1:(10 . 1)
2:(5 . 9)
3:(1 . 3)


()

In [4]:
(define (cc2 amount coin-values)
  (let ((total-list '())
        (coin-list (make-list coin-values)))
    (define (iter a coins l)
      (cond ((= a 0) (begin
                      ; append!手続きがあればスマートかも -> できない。空リストは更新できない。
                      (set! total-list (append total-list (list l)))
                      total-list))
            ((or (< a 0) (no-more? coins)) total-list)
            (else
              (begin
                 (let ((first (first-denomination coins)))
                   (let ((new-l (update-list l (make-pair first 1))))
                     (iter (- a first) coins new-l)
                     )
                   )
                 (iter a (except-first-denomination coins) l)
               )
             )
          )
      )
    (iter amount coin-values coin-list)
  )
)

In [5]:
; 動作確認
(define us-coins (list 50 25 10 5 1))
(define comb-list (cc2 50 us-coins))
(print-list comb-list)
(newline)
(display (length comb-list))
(newline)

1:((50 . 1) (25 . 0) (10 . 0) (5 . 0) (1 . 0))
2:((50 . 0) (25 . 2) (10 . 0) (5 . 0) (1 . 0))
3:((50 . 0) (25 . 1) (10 . 2) (5 . 1) (1 . 0))
4:((50 . 0) (25 . 1) (10 . 2) (5 . 0) (1 . 5))
5:((50 . 0) (25 . 1) (10 . 1) (5 . 3) (1 . 0))
6:((50 . 0) (25 . 1) (10 . 1) (5 . 2) (1 . 5))
7:((50 . 0) (25 . 1) (10 . 1) (5 . 1) (1 . 10))
8:((50 . 0) (25 . 1) (10 . 1) (5 . 0) (1 . 15))
9:((50 . 0) (25 . 1) (10 . 0) (5 . 5) (1 . 0))
10:((50 . 0) (25 . 1) (10 . 0) (5 . 4) (1 . 5))
11:((50 . 0) (25 . 1) (10 . 0) (5 . 3) (1 . 10))
12:((50 . 0) (25 . 1) (10 . 0) (5 . 2) (1 . 15))
13:((50 . 0) (25 . 1) (10 . 0) (5 . 1) (1 . 20))
14:((50 . 0) (25 . 1) (10 . 0) (5 . 0) (1 . 25))
15:((50 . 0) (25 . 0) (10 . 5) (5 . 0) (1 . 0))
16:((50 . 0) (25 . 0) (10 . 4) (5 . 2) (1 . 0))
17:((50 . 0) (25 . 0) (10 . 4) (5 . 1) (1 . 5))
18:((50 . 0) (25 . 0) (10 . 4) (5 . 0) (1 . 10))
19:((50 . 0) (25 . 0) (10 . 3) (5 . 4) (1 . 0))
20:((50 . 0) (25 . 0) (10 . 3) (5 . 3) (1 . 5))
21:((50 . 0) (25 . 0) (10 . 3) (5 . 2) (1 

In [6]:
; 動作確認
(define comb-list2 (cc2 50 (reverse us-coins)))
(print-list comb-list2)
(newline)
(display (length comb-list2))
(newline)

1:((1 . 50) (5 . 0) (10 . 0) (25 . 0) (50 . 0))
2:((1 . 45) (5 . 1) (10 . 0) (25 . 0) (50 . 0))
3:((1 . 40) (5 . 2) (10 . 0) (25 . 0) (50 . 0))
4:((1 . 40) (5 . 0) (10 . 1) (25 . 0) (50 . 0))
5:((1 . 35) (5 . 3) (10 . 0) (25 . 0) (50 . 0))
6:((1 . 35) (5 . 1) (10 . 1) (25 . 0) (50 . 0))
7:((1 . 30) (5 . 4) (10 . 0) (25 . 0) (50 . 0))
8:((1 . 30) (5 . 2) (10 . 1) (25 . 0) (50 . 0))
9:((1 . 30) (5 . 0) (10 . 2) (25 . 0) (50 . 0))
10:((1 . 25) (5 . 5) (10 . 0) (25 . 0) (50 . 0))
11:((1 . 25) (5 . 3) (10 . 1) (25 . 0) (50 . 0))
12:((1 . 25) (5 . 1) (10 . 2) (25 . 0) (50 . 0))
13:((1 . 25) (5 . 0) (10 . 0) (25 . 1) (50 . 0))
14:((1 . 20) (5 . 6) (10 . 0) (25 . 0) (50 . 0))
15:((1 . 20) (5 . 4) (10 . 1) (25 . 0) (50 . 0))
16:((1 . 20) (5 . 2) (10 . 2) (25 . 0) (50 . 0))
17:((1 . 20) (5 . 1) (10 . 0) (25 . 1) (50 . 0))
18:((1 . 20) (5 . 0) (10 . 3) (25 . 0) (50 . 0))
19:((1 . 15) (5 . 7) (10 . 0) (25 . 0) (50 . 0))
20:((1 . 15) (5 . 5) (10 . 1) (25 . 0) (50 . 0))
21:((1 . 15) (5 . 3) (10 . 2)